## Data Modeling

- **Author:** Anthony Bäckström
- **Date:** 16.11.2023
- **Last Update:** 

# **Vielä muistiinpano vaiheessa teksti**
## Tullaan kirjoittamaan puhtaaksi mallintamisen jälkeen

## Data Puhdistusprosessi

### Tavoitteet ja Lähestymistapa

- Tavoite: Puhdistaa datasetti, muuntaa arvot numeerisiksi ja poistaa sellaiset autot, joista puuttuu tietoja.
- Uniikkien Arvojen Säilyttäminen: Pyrimme säilyttämään kaikki uniikit arvot datasetissä.
- Datatiedoston Koon Harventaminen: Tavoitteena on harventaa tiedoston kokoa, mikä mahdollistaa tarkemman datan käytön.

### Dataprosessoinnin Vaiheet

- Ladataan data ja muutetaan dealer_zip-sarakkeen tyyppi merkkijonoksi käyttäen pandas-kirjastoa. Käytämme tqdm-kirjastoa prosessin seurantaan.
- Jos tarpeeksi tarkkoja tuloksia ei saada, harkitaan tyhjien arvojen muuttamista keskiarvojen, mediaanien, moodien avulla tai jopa ennustetaan puuttuvat arvot mallin avulla.
- Valitaan tarvittavat sarakkeet

In [ ]:
import pandas as pd
from tqdm import tqdm

def load_data(file_name):
    df_iter = pd.read_csv(file_name, dtype={"dealer_zip": str}, low_memory=False, iterator=True, chunksize=1000)
    df = pd.concat(tqdm(df_iter, desc="Loading Data"))
    return df

df = load_data("used_cars_data.csv")

In [ ]:
df_cleaned = df.loc[:, [
    'body_type', 'city_fuel_economy', 'highway_fuel_economy', 
    'engine_cylinders', 'exterior_color', 'frame_damaged', 
    'has_accidents', 'salvage', 'fuel_tank_volume', 'fuel_type', 
    'horsepower', 'isCab', 'major_options', 'make_name', 
    'maximum_seating', 'mileage', 'model_name', 'price', 'seller_rating', 
    'torque', 'transmission', 'wheel_system', 'year'
]].copy()

In [ ]:
# Save to new file
df_cleaned.to_csv('new_cars_data.csv', index=False)

## Uuden Polttoaineen Yhdistelmäsarakkeen Luominen

## Tehtävän Kuvaus

- Tehdään uusi sarakkeen 'fuel_comb' luominen, joka on keskiarvo 'city_fuel_economy' ja 'highway_fuel_economy' sarakkeista. Tämän jälkeen poistetaan alkuperäiset sarakkeet.

## Toteutusvaiheet

- Ladataan puhdistettu datasetti.
- Lasketaan uuden 'fuel_comb'-sarakkeen arvot ottamalla keskiarvo 'city_fuel_economy' ja 'highway_fuel_economy' sarakkeista.
- Poistetaan 'city_fuel_economy' ja 'highway_fuel_economy' sarakkeet datasetistä.
- Tallennetaan päivitetty datasetti uudelleen.

In [ ]:
import pandas as pd

# Load the cleaned dataset
file_path = 'new_cars_data.csv'
data_cleaned = pd.read_csv(file_path)

# Calculate the average of 'city_fuel_economy' and 'highway_fuel_economy'
data_cleaned['fuel_comb'] = (data_cleaned['city_fuel_economy'] + data_cleaned['highway_fuel_economy']) / 2

# Drop the original 'city_fuel_economy' and 'highway_fuel_economy' columns
data_cleaned.drop(['city_fuel_economy', 'highway_fuel_economy'], axis=1, inplace=True)

# Save the updated dataset
output_file_path = 'new_cars_data.csv'  
data_cleaned.to_csv(output_file_path, index=False)

print("New column 'fuel_comb' added and original columns removed. Data saved to", output_file_path)

## Tyhjien Arvojen Poistaminen Autoista

### Tehtävän Kuvaus

- Poistetaan kaikki autot datasetistä, joissa on tyhjiä arvoja.
- Data ennen Poistoa: Alkuperäisessä datasetissä oli yli 3 miljoonaa autoa.
- Data Poiston Jälkeen: Poistoprosessin jälkeen datasetissä on yli 1,1 miljoonaa autoa.

### Toteutusvaiheet

- Ladataan datasetti.
- Tarkistetaan ja poistetaan tyhjät arvot eri sarakkeista.
- Tallennetaan puhdistettu datasetti uuteen tiedostoon.

In [ ]:
import pandas as pd

# Load data

df = pd.read_csv('new_cars_data.csv')

columns_to_check = [
    'body_type', 'fuel_comb',
    'engine_cylinders', 'exterior_color', 'frame_damaged',
    'has_accidents', 'salvage', 'fuel_tank_volume', 'fuel_type',
    'horsepower', 'isCab', 'major_options', 'make_name',
    'maximum_seating', 'mileage', 'model_name', 'price', 'seller_rating',
    'torque', 'transmission', 'wheel_system', 'year'
]

for column in columns_to_check:
    # Count the number of missing values in each column
    num_missing_values = df[column].isna().sum()
    if num_missing_values > 0:
        # Remove rows where the column value is missing
        df = df.dropna(subset=[column])
        print(f"Removed {num_missing_values} rows with missing values in '{column}'.")

# Save the cleaned data to a new file

df.to_csv('new_cars_data.csv', index=False)

print("Data cleaned and saved successfully!")

​

    Removed 13543 rows with missing values in 'body_type'.
    Removed 477771 rows with missing values in 'fuel_comb'.
    Removed 10741 rows with missing values in 'engine_cylinders'.
    Removed 24 rows with missing values in 'exterior_color'.
    Removed 1176315 rows with missing values in 'frame_damaged'.
    Removed 5422 rows with missing values in 'fuel_tank_volume'.
    Removed 875 rows with missing values in 'horsepower'.
    Removed 45886 rows with missing values in 'major_options'.
    Removed 13910 rows with missing values in 'mileage'.
    Removed 19519 rows with missing values in 'seller_rating'.
    Removed 112492 rows with missing values in 'torque'.
    Removed 8285 rows with missing values in 'transmission'.
    Removed 91 rows with missing values in 'wheel_system'.
    Data cleaned and saved successfully!


## True/False Sarakkeiden Muuntaminen 1/0 Tyypeiksi

### Tehtävän Kuvaus

- Muuttaa true/false -arvot 1/0 -arvoiksi tietyissä sarakkeissa.

### Toteutusvaiheet

- Ladataan datasetti.
- Tunnistetaan ja muunnetaan tietyt boolean-sarakkeet 1/0 -arvoiksi.
- Tallennetaan muutetut tiedot uuteen tiedostoon.

In [ ]:
import pandas as pd

# Load data

df = pd.read_csv('new_cars_data.csv')

# List of columns to convert from True/False to 1/0
boolean_columns = ['frame_damaged', 'has_accidents', 'salvage', 'isCab']

for column in boolean_columns:
    # Check if the column exists in the DataFrame
    if column in df.columns:
        # Convert True/False to 1/0
        df[column] = df[column].map({True: 1, False: 0})
        print(f"Converted '{column}' from True/False to 1/0.")
    else:
        print(f"Column '{column}' not found in the DataFrame.")

# Save the modified data to a new file

df.to_csv('new_cars_data.csv', index=False)

print("Data modified and saved successfully!")

    Converted 'frame_damaged' from True/False to 1/0.
    Converted 'has_accidents' from True/False to 1/0.
    Converted 'salvage' from True/False to 1/0.
    Converted 'isCab' from True/False to 1/0.
    Data modified and saved successfully!

- arvojen pyöristys kahden desimaalin tarkkuudella

In [ ]:
import pandas as pd

# Load data

df = pd.read_csv('new_cars_data.csv')

# Round the 'seller_rating' column to two decimal places
df['seller_rating'] = df['seller_rating'].round(2)

#  display the first few rows to verify the changes
print(df.head())

# Save the modified data back to a CSV file 
df.to_csv('new_cars_data.csv', index=False)

## Polttoainesäiliön Tilavuuden Muokkaus

### Tehtävän Kuvaus

- Muuttaa 'fuel_tank_volume'-sarakkeen tietoja poistamalla 'gal'-yksikkö ja muuntamalla arvot numeerisiksi.

### Toteutusvaiheet

- Ladataan datasetti.
- Poistetaan 'gal'-yksikkö 'fuel_tank_volume'-sarakkeesta ja muunnetaan jäljelle jäävät arvot numeerisiksi.
- Tallennetaan muutokset ja vahvistetaan ne tarkastelemalla datan ensimmäisiä rivejä.
- Tallennetaan muokatut tiedot takaisin CSV-tiedostoon.

In [ ]:
import pandas as pd


df = pd.read_csv('new_cars_data.csv')

# Remove 'gal' from the 'fuel_tank_volume' column and convert to numeric
df['fuel_tank_volume'] = df['fuel_tank_volume'].str.replace(' gal', '', regex=False)
df['fuel_tank_volume'] = pd.to_numeric(df['fuel_tank_volume'], errors='coerce')

# Display the first few rows to verify the changes
print(df.head())

# Save the modified data back to a CSV file
df.to_csv('new_cars_data.csv', index=False)

## Penkkien Määrän Muokkaus

### Tehtävän Kuvaus

- Muuttaa 'maximum_seating'-sarakkeen tietoja poistamalla sana 'seats' ja muuntamalla arvot numeerisiksi.

### Toteutusvaiheet

- Ladataan datasetti.
- Poistetaan sana 'seats' 'maximum_seating'-sarakkeesta ja muunnetaan jäljelle jäävät arvot numeerisiksi.
- Näytetään ensimmäiset rivit muutosten vahvistamiseksi.
- Tallennetaan muokatut tiedot takaisin CSV-tiedostoon.

In [ ]:
import pandas as pd

# Load data

df = pd.read_csv('new_cars_data.csv')

# Remove 'seats' from the 'maximum_seating' column and convert to numeric
df['maximum_seating'] = df['maximum_seating'].str.replace(' seats', '', regex=False)
df['maximum_seating'] = pd.to_numeric(df['maximum_seating'], errors='coerce').astype('Int64')

# Display the first few rows to verify the changes
print(df.head())


# Save the modified data back to a CSV file
df.to_csv('new_cars_data.csv', index=False)

# objektien muuttaminen koneoppia varten
- luon uuden tiedoston varmuuden vuoksi, jos tulee jotai säbää

## Korin Mallin One-Hot Encoding

### Tehtävän Kuvaus

- Suorittaa one-hot encoding 'body_type'-sarakkeelle binäärimenetelmällä, koska eri kori tyyppien arvoja on vähän.

### Toteutusvaiheet

- Ladataan datasetti.
- Suoritetaan one-hot encoding 'body_type'-sarakkeelle pandas-kirjaston avulla.
- Näytetään ensimmäiset rivit muutosten vahvistamiseksi.
- Tallennetaan muokatut tiedot uuteen CSV-tiedostoon.


In [ ]:
import pandas as pd

# Load dataset
file_path = 'new_cars_data.csv'
df = pd.read_csv(file_path)

# Perform one-hot encoding on the 'body_type' column
df_encoded = pd.get_dummies(df, columns=['body_type'])

# Display the first few rows to verify the changes
print(df_encoded.head())

# Save the modified data 
df_encoded.to_csv('testi.csv', index=False)

## Moottorin Mallin Muokkaus ja One-Hot Encoding

### Tehtävän Kuvaus

- Soveltamalla one-hot encodingia moottorin mallille, jaotellaan 'engine_cylinders'-sarakkeen arvot perusmoottorityypeiksi ja lisäominaisuuksiksi.
- Perusmallit: Esimerkiksi V6, V8, I4 jne.
- Lisäarvot: Flex Fuel, Diesel, Hybrid, Biodiesel, Compressed Natural Gas ja Propane, merkittynä 1/0 menetelmällä.

### Toteutusvaiheet

- Ladataan datasetti.
- Erotellaan 'engine_cylinders'-sarakkeesta perusmoottorityypit ja lisäominaisuudet.
- Suoritetaan one-hot encoding 'Engine_Type'-sarakkeelle.
- Tallennetaan muokatut tiedot uuteen CSV-tiedostoon.

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv('testi.csv')

# Split 'engine_cylinders' into 'Engine Type' and additional attributes
df['Engine_Type'] = df['engine_cylinders'].str.extract(r'([IVWHR]\d+)')

# Additional attributes
df['Flex_Fuel'] = df['engine_cylinders'].str.contains('Flex Fuel Vehicle').astype(int)
df['Diesel'] = df['engine_cylinders'].str.contains('Diesel').astype(int)
df['Hybrid'] = df['engine_cylinders'].str.contains('Hybrid').astype(int)
df['Biodiesel'] = df['engine_cylinders'].str.contains('Biodiesel').astype(int)
df['Compressed_Natural_Gas'] = df['engine_cylinders'].str.contains('Compressed Natural Gas').astype(int)
df['Propane'] = df['engine_cylinders'].str.contains('Propane').astype(int)

# Perform one-hot encoding on the 'Engine_Type' column
df = pd.get_dummies(df, columns=['Engine_Type'])

# Save the modified data to a new CSV file
output_file_path = 'testi.csv'  
df.to_csv(output_file_path, index=False)

print(f"Data modified and saved to {output_file_path}")


## 'Exterior_Color'-sarakkeen Muuntaminen Taajuuskoodauksella

### Tehtävän Kuvaus

- Käsitellä 'exterior_color'-sarakkeen suurta kardinalisuutta taajuuskoodauksen avulla. Tässä menetelmässä ainutlaatuiset värit korvataan niiden esiintymistiheydellä datasetissä.
- Taajuuskoodaus: Yksinkertainen ja tehokas tapa käsitellä monimutkaisia kategorisia muuttujia, jossa korkean kardinalisuuden ongelma minimoidaan.

### Toteutusvaiheet

- Ladataan datasetti.
- Lasketaan kunkin värin esiintymistiheys.
- Korvataan värit niiden tiheyksillä 'exterior_color'-sarakkeessa.
- Käytetään muunnettua ominaisuutta koneoppimisessa.
- Tallennetaan muokatut tiedot tarvittaessa uuteen CSV-tiedostoon.

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv('testi.csv')

# Calculate the frequency of each color
color_frequency = df['exterior_color'].value_counts(normalize=True)  # normalize=True gives the proportion

# Map each color to its frequency
df['exterior_color'] = df['exterior_color'].map(color_frequency)

# Display the first few rows to verify the changes
print(df.head())


df.to_csv('testi.csv', index=False)

## 'Fuel_Type'-sarakkeen Muuntaminen One-Hot Encodingilla

### Tehtävän Kuvaus

- Muuntaa 'fuel_type'-sarakkeen kategoriset arvot one-hot encode -muotoon, mikä on hyödyllistä koneoppimismalleja varten.

### Toteutusvaiheet

- Ladataan datasetti.
- Suoritetaan one-hot encoding 'fuel_type'-sarakkeelle.
- Tarkistetaan muutokset näyttämällä datan ensimmäiset rivit.
- Tallennetaan muokatut tiedot tarvittaessa uuteen CSV-tiedostoon.

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv('testi.csv')

# Perform one-hot encoding on the 'fuel_type' column
df_encoded = pd.get_dummies(df, columns=['fuel_type'])

# Display the first few rows to verify the changes
print(df_encoded.head())


df_encoded.to_csv('testi.csv', index=False)

## Lisävarusteiden Määrän Laskeminen Autossa

### Tehtävän Kuvaus

- Laskea kunkin auton lisävarusteiden määrä 'major_options'-sarakkeesta, joka on lista muodossa.
- Haaste: 'Major_options'-sarake sisältää monia uniikkeja arvoja, jotka tekevät datan puhdistamisesta haastavaa.

### Toteutusvaiheet

- Ladataan datasetti.
- Luodaan funktio, joka laskee jokaisen auton lisävarusteiden määrän 'major_options'-sarakkeesta.
- Sovelletaan funktiota ja lisätään tulos uutena sarakkeena datasettiin.
- Tarkistetaan muutokset näyttämällä datan ensimmäiset rivit.
- Tallennetaan muokatut tiedot tarvittaessa uuteen CSV-tiedostoon.

In [ ]:
%pip install pandas

import pandas as pd
import ast  # This will be used to safely evaluate the string representation of lists

# Load data
df = pd.read_csv('testi.csv')

# Function to count the number of options
# Assumes that every entry in 'major_options' is a valid list of options
def count_options(option_string):
    try:
        options_list = ast.literal_eval(option_string)
        return len(options_list)
    except:
        return 0  # Returns 0 in case of any error in parsing

# Apply the function to the 'major_options' column
df['options_count'] = df['major_options'].apply(count_options)

# Display the first few rows to verify the changes
print(df.head())


df.to_csv('testi.csv', index=False)

## Automerkeille Oman Sarakkeen Luominen One-Hot Encodingilla

### Tehtävän Kuvaus

- Suorittaa one-hot encoding 'make_name'-sarakkeelle, luomalla jokaiselle automerkille oman sarakkeen.

### Toteutusvaiheet

- Ladataan datasetti.
- Suoritetaan one-hot encoding 'make_name'-sarakkeelle.
- Tarkistetaan muutokset näyttämällä datan ensimmäiset rivit.
- Tallennetaan muokatut tiedot tarvittaessa uuteen CSV-tiedostoon.

In [ ]:
%pip install pandas
import pandas as pd

# Load data
df = pd.read_csv('testi.csv')

# Perform one-hot encoding on the 'make_name' column
df_encoded = pd.get_dummies(df, columns=['make_name'])

# Display the first few rows to verify the changes
print(df_encoded.head())


df_encoded.to_csv('testi.csv', index=False)

## Automerkeille Oman Sarakkeen Luominen One-Hot Encodingilla

### Tehtävän Kuvaus

- Suorittaa one-hot encoding 'make_name'-sarakkeelle, luomalla jokaiselle automerkille oman sarakkeen.

### Toteutusvaiheet

- Ladataan datasetti.
- Suoritetaan one-hot encoding 'make_name'-sarakkeelle.
- Tarkistetaan muutokset näyttämällä datan ensimmäiset rivit.
- Tallennetaan muokatut tiedot tarvittaessa uuteen CSV-tiedostoon.

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv('testi.csv')

# Calculate the frequency of each model name
model_frequency = df['model_name'].value_counts(normalize=True)  # normalize=True gives the proportion

# Map each model name to its frequency
df['model_name'] = df['model_name'].map(model_frequency)

# Display the first few rows to verify the changes
print(df.head())

df.to_csv('testi.csv', index=False)

## 'Transmission'-sarakkeen Muuntaminen One-Hot Encodingilla

### Tehtävän Kuvaus

- Suorittaa one-hot encoding 'transmission'-sarakkeelle, luomalla jokaiselle vaihteistotyypille oman sarakkeen.

### Toteutusvaiheet

- Ladataan datasetti.
- Suoritetaan one-hot encoding 'transmission'-sarakkeelle.
- Tarkistetaan muutokset näyttämällä datan ensimmäiset rivit.
- Tallennetaan muokatut tiedot tarvittaessa uuteen CSV-tiedostoon.

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv('testi.csv')

# Perform one-hot encoding on the 'transmission' column
df_encoded = pd.get_dummies(df, columns=['transmission'])

# Display the first few rows to verify the changes
print(df_encoded.head())

df_encoded.to_csv('testi.csv', index=False)

## 'Wheel_System'-sarakkeen Muuntaminen One-Hot Encodingilla

### Tehtävän Kuvaus

- Suorittaa one-hot encoding 'wheel_system'-sarakkeelle, luomalla jokaiselle pyöräjärjestelmätyypille oman sarakkeen.

### Toteutusvaiheet

- Ladataan datasetti.
- Suoritetaan one-hot encoding 'wheel_system'-sarakkeelle.
- Tarkistetaan muutokset näyttämällä datan ensimmäiset rivit.
- Tallennetaan muokatut tiedot uuteen CSV-tiedostoon nimellä 'testi3.csv'.

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv('testi.csv')

# Perform one-hot encoding on the 'wheel_system' column
df_encoded = pd.get_dummies(df, columns=['wheel_system'])

# Display the first few rows to verify the changes
print(df_encoded.head())

df_encoded.to_csv('testi.csv', index=False)

## Koneoppimallien testaus